In [1]:
!pip install pandas
!pip install scikit-learn

!pip install bitsandbytes
!pip install accelerate

import torch

In [134]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# ekperimentas 1
# llama3 modelis geras lietuviu kalbai
# suskaiciuoti tikslumas, darant klasifikacija
# use prompts/instrukcija
https://www.kaggle.com/code/lucamassaron/fine-tune-llama-3-for-sentiment-analysis
# modelio pacio nekeiciam ir nemokinam, tsg jam duodam duomenis ir jis klasifikuoja kaip moka

# ekperimentas 2
# transfer learning arba fine-tuning
https://huggingface.co/docs/transformers/tasks/sequence_classification
# cia vyks mokymas

# galesim palygint situos 2 atvejus ir palygint ju statistikas, pvz tikslumas, F1 ir t.t.

# ekperimentas 3
# gebetu identifikuot kur yra konkretus fta pastraipoje
# rodyti indeksus kur prasidejo ir baigesi, jeigu nfta tai 0, 0
# reiks keist exceli, kad rodyt pradzios intervala ir pabaigos

# ekperimentas 4
# naudot lietuviska llama2 kuris jau pamokytas lietuviu kalbai
# ...

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

file_path = 'FTA_dataset.xlsx'
sheet1 = pd.read_excel(file_path, sheet_name='FTA', header=None)
sheet2 = pd.read_excel(file_path, sheet_name='NFTA', header=None)

sheet1.columns = ['paragraph']
sheet2.columns = ['paragraph']
sheet1['fta'] = 1
sheet2['fta'] = 0

data = pd.concat([sheet1, sheet2], ignore_index=True)

# Shuffle the data
# sample() - randomly select data
# frac=1 - select 100% of data
# After shuffling, the original index of the DataFrame is no longer in order
# reset_index(drop=True) is used to reset the index to a new sequential order.
# drop=True - discards the old index
data = data.sample(frac=1).reset_index(drop=True)

# training (70%), validation (10%), and test (20%)
# saugoti juos i excel failus just in case kitiem ekperimentams !
train, temp = train_test_split(data, test_size=0.3, random_state=42)
val, test = train_test_split(temp, test_size=(2/3), random_state=42)

Eksperimentas 1

In [72]:
y_true = test["fta"].tolist()

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

tokenizer = AutoTokenizer.from_pretrained("Undi95/Meta-Llama-3-8B-hf")
model = AutoModelForCausalLM.from_pretrained("Undi95/Meta-Llama-3-8B-hf",
                                             load_in_8bit=True,
                                             device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [127]:
def generate_prompt(data_point):
    return f"""
            Sociolingvistikos srityje Veido Grasinantis Veiksmas (VGV) yra komunikacinis veiksmas, kuris kelia grėsmę kalbėtojo ar klausytojo socialinei tapatybei ir savivertės jausmui.
            Pavyzdžiai VGV yra įžeidimai, pertraukimai ir kritika.
            Išanalizuokite šį naujienų tekstą ir atsakykite, ar jame yra Veido Grasinantis Veiksmas (VGV).
            Atsakykite „1“ už „taip“ arba „0“ už „ne“.

            [{data_point["paragraph"]}] = """

In [128]:
X_test = pd.DataFrame(test.apply(generate_prompt, axis=1), columns=["paragraph"])

In [129]:
from tqdm import tqdm

def predict(model, tokenizer):
    y_pred = []
    pipe = pipeline(task="text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens = 1,
                do_sample=False
                )
    for i in tqdm(range(len(X_test))):
        prompt = X_test.iloc[i]["paragraph"]
        result = pipe(prompt)
        answer = result[0]['generated_text'].split("=")[-1]
        print("prompt: ", prompt)
        print("answer: ", answer)
        if "1" in answer:
            y_pred.append(1)
        elif "0" in answer:
             y_pred.append(0)
        else:
            y_pred.append(-1)
    return y_pred

In [130]:
y_pred = predict(model, tokenizer)

  0%|          | 0/143 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  1%|          | 1/143 [00:00<01:35,  1.48it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


prompt:  
            Sociolingvistikos srityje Veido Grasinantis Veiksmas (VGV) yra komunikacinis veiksmas, kuris kelia grėsmę kalbėtojo ar klausytojo socialinei tapatybei ir savivertės jausmui. 
            Pavyzdžiai VGV yra įžeidimai, pertraukimai ir kritika. 
            Išanalizuokite šį naujienų tekstą ir atsakykite, ar jame yra Veido Grasinantis Veiksmas (VGV). 
            Atsakykite „1“ už „taip“ arba „0“ už „ne“.

            [Kad nebūtų vien tiktai tuščios kalbos, aš ir mano taip pat vadovaujamo sąrašo nariai patys nuėjom nusiėmėm verslo liudijimus, pradėjome dirbti „Uberio“ taksi vairuotojais, tvarkyti butus, prekiauti kava ir mes labai aiškiai matome, kur yra šiandien realios problemos, kodėl mūsų jaunimas arba pradedantieji verslininkai nenori imtis atakomybės ir pradėti verslą. Tai todėl mūsų programoj yra labai aiškios priemonės, atėjusios jau iš konkrečios praktikos ir po diskusijų su specialistais. Pradedančiajam verslui iki penkių tūkstančių eurų metinių pajamų – nu

  1%|▏         | 2/143 [00:01<01:22,  1.72it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


prompt:  
            Sociolingvistikos srityje Veido Grasinantis Veiksmas (VGV) yra komunikacinis veiksmas, kuris kelia grėsmę kalbėtojo ar klausytojo socialinei tapatybei ir savivertės jausmui. 
            Pavyzdžiai VGV yra įžeidimai, pertraukimai ir kritika. 
            Išanalizuokite šį naujienų tekstą ir atsakykite, ar jame yra Veido Grasinantis Veiksmas (VGV). 
            Atsakykite „1“ už „taip“ arba „0“ už „ne“.

            [Taip, tiesiog norėjau papildyti, kad, iš tiesų, reikia sudaryti sąlygas žmonėms dirbti regionuose. Tai ne vien tik miestai tuštėja, tokie regioniniai, vadinkime, bet ir, ypatingai, kaimai, ir ten yra galimybių, yra pasiūlymų, kaip tai galima gaivinti kaimą.] = 
result:  [{'generated_text': '\n            Sociolingvistikos srityje Veido Grasinantis Veiksmas (VGV) yra komunikacinis veiksmas, kuris kelia grėsmę kalbėtojo ar klausytojo socialinei tapatybei ir savivertės jausmui. \n            Pavyzdžiai VGV yra įžeidimai, pertraukimai ir kritika. \n       

  2%|▏         | 3/143 [00:01<01:05,  2.14it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

prompt:  
            Sociolingvistikos srityje Veido Grasinantis Veiksmas (VGV) yra komunikacinis veiksmas, kuris kelia grėsmę kalbėtojo ar klausytojo socialinei tapatybei ir savivertės jausmui. 
            Pavyzdžiai VGV yra įžeidimai, pertraukimai ir kritika. 
            Išanalizuokite šį naujienų tekstą ir atsakykite, ar jame yra Veido Grasinantis Veiksmas (VGV). 
            Atsakykite „1“ už „taip“ arba „0“ už „ne“.

            [Na, ir dar pridėčiau, kad, iš tikrųjų, dviguba pilietybė ir elektroninis balsavimas yra tie sprendimai, kurie yra būtini ir tikrai nesuprantu kolegų konservatorių, kodėl jie priešinasi būtent elektroniniam balsavimui. Na, o mes turime lygiai taip pat būt ir atvira visuomenė, eem, vienas dalykas yra kalbėt apie migraciją, bet aš esu įsitikinęs, kad Lietuva gali tapti šalis, regiono lyderė, pritraukiant, na, kūrybingus, išsilavinusius jaunuolius iš užsienio šalių ir būtent tie mūsų emigrantai gali būt puikus tarpininkas tokiam ambicingam tikslui siekti.]

 99%|█████████▉| 142/143 [01:13<00:00,  1.92it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


prompt:  
            Sociolingvistikos srityje Veido Grasinantis Veiksmas (VGV) yra komunikacinis veiksmas, kuris kelia grėsmę kalbėtojo ar klausytojo socialinei tapatybei ir savivertės jausmui. 
            Pavyzdžiai VGV yra įžeidimai, pertraukimai ir kritika. 
            Išanalizuokite šį naujienų tekstą ir atsakykite, ar jame yra Veido Grasinantis Veiksmas (VGV). 
            Atsakykite „1“ už „taip“ arba „0“ už „ne“.

            [Aš noriu, kad kalbėtų mokytojai, kurie jaunimą pažįsta, jį uždo, Dainora Eigminienė, lituanistė, Vilniaus Kristoforo gimnazijos mokytoja, prašom.] = 
result:  [{'generated_text': '\n            Sociolingvistikos srityje Veido Grasinantis Veiksmas (VGV) yra komunikacinis veiksmas, kuris kelia grėsmę kalbėtojo ar klausytojo socialinei tapatybei ir savivertės jausmui. \n            Pavyzdžiai VGV yra įžeidimai, pertraukimai ir kritika. \n            Išanalizuokite šį naujienų tekstą ir atsakykite, ar jame yra Veido Grasinantis Veiksmas (VGV). \n          

100%|██████████| 143/143 [01:13<00:00,  1.94it/s]

prompt:  
            Sociolingvistikos srityje Veido Grasinantis Veiksmas (VGV) yra komunikacinis veiksmas, kuris kelia grėsmę kalbėtojo ar klausytojo socialinei tapatybei ir savivertės jausmui. 
            Pavyzdžiai VGV yra įžeidimai, pertraukimai ir kritika. 
            Išanalizuokite šį naujienų tekstą ir atsakykite, ar jame yra Veido Grasinantis Veiksmas (VGV). 
            Atsakykite „1“ už „taip“ arba „0“ už „ne“.

            [Aš tik norėjau sureaguoti, kad, matot, jeigu mokytojas rūko, tai kiek jis padaro nuostolio, nes mūsų daug jaunimo, žinot, išmirs ir tam tikras šešėlis ir šita pramonė labai klesti. Aš kada pats Pakuonį būdamas kviečiau surinkti pakučius ir rasti, tuos išmestus, kiek yra iš vietinės pramonės. Ir man, žinote, tas šešėlis ne tai, kad jis padaro nuostolį ten finansais, bet jis kiek nušluoja žmonių gyvybių, nes, žinot, pasižiūrėta mokykloj kaip mes kaip keliolikos metų vaikai jau drūčiai prasirūkę. Tai tas šešėlis atneša daugeliui mirtį ir ypatingai tos vis

In [131]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate(y_true, y_pred):
    # Filter out undetermined predictions
    mask = [y != -1 for y in y_pred]
    filtered_y_true = [y_true[i] for i in range(len(y_true)) if mask[i]]
    filtered_y_pred = [y_pred[i] for i in range(len(y_pred)) if mask[i]]

    accuracy = accuracy_score(filtered_y_true, filtered_y_pred)
    precision = precision_score(filtered_y_true, filtered_y_pred, pos_label=1)
    recall = recall_score(filtered_y_true, filtered_y_pred, pos_label=1)
    f1 = f1_score(filtered_y_true, filtered_y_pred, pos_label=1)

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")

In [132]:
evaluate(y_true, y_pred)

Accuracy: 0.3076923076923077
Precision: 0.25196850393700787
Recall: 0.8888888888888888
F1 Score: 0.39263803680981596


Eksperimentas 2